# <center> Майнор "Интеллектуальный анализ данных" </center>

# <center> Курс "Введение в программирование" </center>

# <center> Визуализация и первичный анализ данных </center>

## Table of contents

1. [Links](#links)
2. [Визуализация в Python](#vis)
    1. [Визуализация динамики](#dynamics)
    2. [Распределение дискретных признаков](#discrete)
    3. [Распределение непрерывных признаков](#contin)
        1. [Гистограмма](#hist)
        2. [Boxplot](#boxplot)
    4. [Визуализация зависимости](#depend)
        1. [Scatterplot](#scatter)
        2. [Heatmap](#heat)
    5. [Несколько графиков на одном](#subplot)
    6. [Сохранение изображения](#save)
    7. [Подписи осей](#labels)

## Links <a name="links"></a>

 - [Вторая статья](https://habrahabr.ru/company/ods/blog/323210/) из открытого курса ODS по машинному обучению, посвященная визуализации<br><br>
 - [Глава](https://pandas.pydata.org/pandas-docs/stable/visualization.html#) про визуализацию в Pandas в документации библиотеки<br><br>
 - [Официальный сайт](https://seaborn.pydata.org/index.html) seaborn, на котором есть документация, тьюториалы и галерея графиков<br><br>
 - Неплохой [тьюториал](https://elitedatascience.com/python-seaborn-tutorial) по seaborn<br><br>
 - [Галерея](https://python-graph-gallery.com) разнообразных графиков на python с кодом. В частности, небольшой [список](https://python-graph-gallery.com/bad-chart/) ошибок при визуализации и примеры, как их можно избежать<br><br> 

## Визуализация в Python <a name="vis"></a>

Сначала выполним все необходимые настройки и подключим библиотеки.  
Для визуализации мы будем использовать: 
 - функционал Pandas - обертка над matplotlib;
 - библиотеку `seaborn` - также надстройка над matplotlib, предоставляющая более приятный интерфейс;
 - библиотеку matplotlib

In [ ]:
import numpy as np
import pandas as pd

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# стиль seaborn
# style.available выводит все доступные стили
from matplotlib import style
style.use('seaborn')

#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

In [ ]:
# увеличим дефолтный размер графиков
from matplotlib import rcParams
rcParams['figure.figsize'] = [8, 5]

In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

Будем практиковаться на наборе данных о продажах и оценках видеоигр: [Kaggle Dataset](https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings).  
  
Описание с источника:   
Alongside the fields: Name, Platform, Year_of_Release, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales, we have: 
- Critic_score - Aggregate score compiled by Metacritic staff
- Critic_count - The number of critics used in coming up with the Critic_score
- User_score - Score by Metacritic's subscribers
- User_count - Number of users who gave the user_score
- Developer - Party responsible for creating the game
- Rating - The [ESRB](https://www.esrb.org/) ratings  
  
Загрузим данные, определим размерность и посмотрим на сами данные

In [ ]:
df = pd.read_csv("video_games_sales.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

Первое, что бросается в глаза - пропуски в данных (`NaN` в ячейках).  
Видно, что пропуски присутствуют в половине признаков. Для простоты удалим такие наблюдения.

In [ ]:
df = df.dropna()

In [ ]:
df.info()

Объем данных заметно уменьшился.  
Теперь посмотрим на количество уникальных игр в датасете.  
Спойлер: их заметно меньше, чем число наблюдений. Сразу найдем все игры, которые встречаются более одного раза, и посмотрим на одну из них в качестве примера.

In [ ]:
df.Name.nunique()

In [ ]:
names = df.Name.value_counts()
names[names > 1]

In [ ]:
df[df.Name == 'Need for Speed: Most Wanted']

Судя по данным, одна и та же игра может быть представлена на различных платформах, и также могут быть перезапуски (более поздний год).  
  
Поменяем типы некоторых признаков перед основной работой, т.к. при чтении они были определены неверно с нашей точки зрения.

In [ ]:
df.info()

In [ ]:
df['User_Score'] = df.User_Score.astype('float64')
df['Year_of_Release'] = df.Year_of_Release.astype('int64')
df['User_Count'] = df.User_Count.astype('int64')
df['Critic_Count'] = df.Critic_Count.astype('int64')

In [ ]:
df.describe()

In [ ]:
df.describe(include=['object'])

### Визуализация динамики <a name="dynamics"></a>

Для визуализации динамики лучше всего подходит стандартный `lineplot`.  
Метод `plot()` в Pandas строит график для каждого столбца, разделяя их по цветам, и добавляет легенду. Удобно. 

In [ ]:
# Просто все признаки, отвечающие за продажи
# В данном случае, они содержат в имени Sales
[x for x in df.columns if 'Sales' in x]

In [ ]:
sales_df = df[[x for x in df.columns if 'Sales' in x] + ['Year_of_Release']]
sales_df.groupby('Year_of_Release').sum().plot()
plt.show()

Если признаков много, на одном графике они могут смешаться в кучу и получится `spaghetti plot`, на котором очень сложно что-то увидеть.  В таких случаях имеет смысл отобразить каждый признак на отдельном подграфе.  

In [ ]:
sales_df.groupby('Year_of_Release').sum().plot(subplots=True, 
                                               layout=(3, 2), 
                                               sharex=True, 
                                               sharey=True,
                                               figsize=(12,8),
                                               linestyle='-', 
                                               marker='o')
plt.show()

### Задание

**Постройте график или несколько, показывающий число игр, выпущенных в каждый год в общем и с разбивкой:**  
  
&nbsp;&nbsp;&nbsp;&nbsp;**- по жанру**  
&nbsp;&nbsp;&nbsp;&nbsp;**- по возрастному рейтингу**

In [ ]:
# ваш код здесь



### Задание

**Провизуализируйте динамику по годам средней оценки пользователей в общем и с разбивкой:**  
  
&nbsp;&nbsp;&nbsp;&nbsp;**- по жанру**  
&nbsp;&nbsp;&nbsp;&nbsp;**- по возрастному рейтингу**

In [ ]:
# ваш код здесь



### Распределение дискретных признаков <a name="discrete"></a>

Для визуализации распределения категориальных признаков хорошо подходит `barplot`. По сути, для каждой категории он строит столбик (bar) высотой равной количеству наблюдений, в которых это значение встретилось. 

In [ ]:
# rot отвечает за угол наклона подписей значений на оси x
df.Genre.value_counts().plot(kind='bar', rot=45) 
plt.show()

Вместо bar'ов можно построить `lollipop plot`.

In [ ]:
n_obs = df.Genre.value_counts()
genres = n_obs.index.values
plt.stem(n_obs)
plt.xticks(range(len(genres)), genres, rotation=45)
plt.show()

Популярной альтернативой `barplot` является `pieplot`. Однако этот тип графика скорее вреден, т.к. он искажает представление о соотношении долей категорий (так говорят). В данном случае еще и цвета у категорий совпадают.

In [ ]:
df.Genre.value_counts().plot(kind='pie')
plt.show()

Можно немного модифицировать график, превратив его в `doughnut plot`. Стало немного лучше, но кардинально ничего не изменилось.  
  
В общем, посыл такой - не использовать `pieplot` и его вариации.

In [ ]:
df.Genre.value_counts().plot(kind='pie')
plt.gcf().gca().add_artist(plt.Circle((0,0), 0.7, color='white'))
plt.show()

Очень удобное свойство `barplot` - на графике можно посмотреть распределение сразу по нескольким признакам. Ниже в примере показаны продажи для каждого жанра. 

In [ ]:
df[[x for x in df.columns if 'Sales' in x] + ['Genre']].groupby('Genre').sum().plot(kind='bar', rot=45)
plt.show()

В таком виде анализировать график тяжело. Как и в случае с `lineplot`, удобнее разбить его на несколько графиков.  
В таком виде информация представлена нагляднее. Например, можно сразу заметить, что продажи по жанрам распределены примерно одинаково везде, кроме Японии - там проявляют особый интерес к RPG и не любят шутеры.

In [ ]:
df[[x for x in df.columns if 'Sales' in x] + ['Genre']].groupby('Genre').sum().plot(subplots=True, 
                                                                                    layout=(3, 2), 
                                                                                    kind='bar', 
                                                                                    rot = 45,
                                                                                    figsize=(12,8))
plt.show()

С помощью метода `transpose()`  можно транспонировать датафрейм, т.е. поменять местами оси.

In [ ]:
df[[x for x in df.columns if 'Sales' in x] + 
   ['Genre']].groupby('Genre').sum().transpose().plot(subplots=True, 
                                                      layout=(6, 2), 
                                                      kind='bar', 
                                                      rot = 45,
                                                      figsize=(12,8))
plt.show()

Для построения `barplot`  мы использовали посчитанные значения для каждой категории. Фактически, мы передавали методу `plot()` индексированный массив.  
  
Функции `countplot()` и `factorplot()` из библиотеки seaborn позволяют передавать им обычный набор данных, и берут на себя подсчет числа наблюдений для каждой категории.  
Для примера посмотрим на число наблюдений с пропущенными значениями в зависимости от жанра или платформы.

In [ ]:
df_orig = pd.read_csv("video_games_sales.csv")

In [ ]:
print(sum(pd.isna(df_orig.Genre)))
print(sum(pd.isna(df_orig.Name)))

In [ ]:
df_orig[pd.isnull(df_orig.Name)]

In [ ]:
df_new = df_orig.dropna(subset=['Name', 'Genre'], how='any')
np.shape(df_new)

In [ ]:
# Создаем новый признак nan_exist со значениями True и False если для наблюдения соотвественно есть или нет 
# пропущенные значения хотя бы по одному признаку
df_new['nan_exist'] = df_new.drop(['Name', 'Genre'], axis=1).isnull().any(axis=1)

In [ ]:
sns.factorplot(x='Genre', hue='nan_exist', data=df_new, kind='count', size = 8).set_xticklabels(rotation=45)
plt.show()

In [ ]:
print('\nMissing values in Platform:', sum(pd.isnull(df_new.Platform)), '\n\n')

plt.figure(figsize=(11.7, 8.27))
sns.countplot(x='Platform', hue='nan_exist', data=df_new)
plt.xticks(rotation=45)
plt.show() # при таком выводе не выводится лишняя информация от matplotlib

По умолчанию категории упорядочены по тому, какая раньше встретилась в наборе данных. Порядок можно поменять с помощью аргумента `order`.  
Можно, например, отсортировать в лексикографическом порядке. 

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(x='Platform', hue='nan_exist', data=df_new, order=sorted(df_new.Platform.unique()))
plt.xticks(rotation=45)
plt.show()

Или в случайном порядке:

In [ ]:
from random import shuffle
x = df_new.Platform.unique()
plt.figure(figsize=(8,5))
shuffle(x) # изменяет объект (перемешивает элементы), возвращает None
sns.countplot(x='Platform', hue='nan_exist', data=df_new, order=x)
plt.xticks(rotation=45)
plt.show()

Или по убыванию величины категории. Но использовать для этого `countplot` не всегда разумно, т.к. все равно приходится вызывать `value_counts()`.

In [ ]:
sns.countplot(x='Platform', hue='nan_exist', data=df_new, 
              order=df_new.Platform.value_counts().index)
plt.xticks(rotation=45)
plt.show()

### Задание
  
**Постройте график или несколько, показывающий число игр, выпущенных в каждый год в общем и с разбивкой:**  
  
&nbsp;&nbsp;&nbsp;&nbsp;**- по жанру**  
&nbsp;&nbsp;&nbsp;&nbsp;**- по возрастному рейтингу**    

**Выполните задание, используя `barplot`.**

In [ ]:
# ваш код здесь



## Распределение непрерывных признаков <a name="contin"></a>

### Гистограмма <a name="hist"></a>

Гистограмма позволяет оценить распределение непрерывной величины, разбивая интервал значений на несколько интервалов и считая, сколько наблюдений попало в каждый из интервалов, аналогично `bar plot`.  
Для правильной оценки распределения важно выбрать число интервалов (они, как правило, равны). Чаще всего по умолчанию используют число интервалов равное $\sqrt{x_{max}}$, где $x_{max}$ - наибольшее наблюдаемое значение.

In [ ]:
df.Critic_Score.hist(figsize=(12,8))
plt.show()

In [ ]:
plt.figure(figsize=(11.7, 8.27))
'''
    Аргумент kde (по умолчанию True) указывает, что мы строим плотность распределения на основе гистограммы
    Если изменить на False, будет просто гистограмма
    Аргумент rug (False by default) отвечает за отображение наблюдений ('палочки' снизу графика)
'''
sns.distplot(df['Critic_Score'], kde=True, rug=True)
plt.show()

Можно построить гистрограмму признака с разбивкой по группам, что позволит оценить распределение признака в каждой группе и сравнить между собой. Но т.к. границы значений наблюдений в каждой группе могут отличаться, гистограммы также будут построены с разными параметрами - числом столбцов и их шириной.

In [ ]:
# разбиение по умолчанию
df[['Critic_Score']].hist(by=df.Genre, figsize=(12,8), sharex=True)
plt.show()

Можно явно задать число столбцов в гистограмме (`bins`). Тем не менее, они будут по-прежнему отличаться шириной.

In [ ]:
# фиксированное число разбиений
df[['Critic_Score']].hist(by=df.Genre, figsize=(12,8), sharex=True, bins=20)
plt.show()

Также можно арументу `bins` вместо числа столбцов передать сами интервалы - массив из n+1 значений, где n - число интервалов. При этом, интервалы могут быть разного размера.  
В таком случае гистограммы для разных групп будут синхронизированы.  
  
Гистограммы позволяют оценить распределение в целом, сделать выводы о принаджлежности распределения к одному из известных, и заметить отклонения. Например, на графиках распределения напоминают нормальное, но средние значения сильно смещены вправо, при этом распределения имеют длинные левые хвосты.  

In [ ]:
# фиксированные интервалы
# от 0 до максимального значения признака с шагом 5
# +1 для того, чтобы максимальное значение вошло в интервал
df[['Critic_Score']].hist(by=df.Genre, 
                          figsize=(12,8), 
                          sharex=True,
                          bins=np.array(range(0,max(df.Critic_Score.astype('int64')) + 1, 5)))
plt.show()

### Задание
  
__Постройте гистрограмму числа пользователей.<br> Также постройте с разбивкой по возрастному рейтингу. Используйте одни и те же интервалы в каждой из гистограмм.__ 

In [ ]:
# ваш код здесь



### Boxplot <a name="boxplot"></a>

Для более детальной оценки и визуализации статистик распределения непрерывных величин принято использовать `boxplot`.  
  
Ящик содержит в себе половину наблюдений - от 0.25 персентиля до 0.75 персентиля. Черта посередине - медиана. Границы "усов" определяются по формуле на изображении ниже. Все, что выходит за эти ганицы, считается выбросами и отображается отдельными точками. Изображение ниже демонстрируем `boxplot` для нормального распределения. 

![title](boxplot.png)

Так можно загрузить и вывести изображение в python 
```python
from scipy import misc
from matplotlib import cm
image = misc.imread('boxplot.png')
plt.figure(figsize = (15,10))
plt.imshow(image, cmap=cm.binary)
```

Ниже представлены боксплоты для оценок критиков с разбивкой по жанрам (аналогично гистограммам).   
Можно сказать, что оценки у спортивных игр в среднем выше остальных жанров, и распределены более плотно. Хуже всего с оценками у приключенческих игр.

In [ ]:
plt.figure(figsize=(11.7, 8.27))
sns.boxplot(y="Genre", x="Critic_Score", data=df, orient="h")
plt.show()

При построении боксплотов мы хорошо визуализируем статистики, но теряем информацию о количестве наблюдений, на которых эти статистики оценивались. Для устранения этого недостатка можно воспользоваться способом, представленным ниже. Также для этого есть модифицированный боксплот - `violin plot`.

In [ ]:
plt.figure(figsize=(11.7, 8.27))
sns.boxplot(y="Genre", x="Critic_Score", data=df, orient="h")
sns.stripplot(y="Genre", x="Critic_Score", data=df, color="orange", jitter=0.2, size=2.5)

### Задание
  
__С помощью `boxplot` оцените распределение оценок пользователей для 7 самых популярных (больше всего пользователей) издателей (publisher).__

In [ ]:
# ваш код здесь



## Визуализация зависимости <a name="depend"></a>

### Scatter plot <a name="scatter"></a>

Диаграмма рассеяния, по сути, визуализирует наблюдения в пространстве двух признаков. На основе распределения точек на графике можно сделать вывод, например, о зависимости признаков. 

In [ ]:
df.plot.scatter(x='User_Score', y='Critic_Score', figsize=(11.7, 8.27))
plt.show()

На график можно добавить информацию о третьем признаке, например, через размер точек.

In [ ]:
df.plot.scatter(x='User_Score', y='Critic_Score', s=df.Global_Sales, figsize=(11.7, 8.27))
plt.show()

Или через цвет.

In [ ]:
sns.pairplot(x_vars='User_Score', y_vars='Critic_Score', data=df, hue="Rating", size=8)
plt.show()

Warning: ниже тяжелый график, может долго отрисовываться и съесть много ресурсов. 

In [ ]:
# sns.pairplot(df[['NA_Sales', 
#                 'EU_Sales', 
#                 'JP_Sales', 
#                 'Other_Sales', 
#                 'Global_Sales', 
#                 'Critic_Score', 
#                 'Critic_Count', 
#                 'User_Score', 
#                 'User_Count']])

## Heatmap <a name="heat"></a>

Тепловые карты просто раскрашивают матрицу числовых значений, используя цветовую шкалу, построенную на основе значений в матрице. Так очень удобно оценивать корреляции между признаками. 

In [ ]:
corr_mat = df[['NA_Sales', 
                 'EU_Sales', 
                 'JP_Sales', 
                 'Other_Sales', 
                 'Global_Sales', 
                 'Critic_Score', 
                 'Critic_Count', 
                 'User_Score', 
                 'User_Count']].corr()

In [ ]:
sns.heatmap(corr_mat, annot=True)
plt.ylim((corr_mat.shape[0], 0)) # fixes bug in matplotlib 3.1.1, should be removed in other versions
plt.show()

Также можно привизуализировать и сводные таблицы.

In [ ]:
sns.heatmap(df.pivot_table(index='Platform', 
                           columns='Genre', 
                            values='User_Score', 
                            aggfunc=np.mean), 
            annot=True, fmt=".1f", linewidths=.5)
plt.ylim((df['Platform'].nunique(), 0)) # fixes bug in matplotlib 3.1.1, should be removed in other versions
plt.show()

## Несколько графиков на одном <a name="subplot"></a>

Просто демонстрация возможностей.

In [ ]:
sns.distplot(np.random.normal(size=1000, scale=1.0), hist=False, label = "std = 1.0")
sns.distplot(np.random.normal(size=1000, scale=2.0), hist=False, label = "std = 2.0")
sns.distplot(np.random.normal(size=1000, scale=3.0), hist=False, label = "std = 3.0")
plt.legend(loc='best')
plt.show()

In [ ]:
plt.subplot(2, 1, 1)
sns.distplot(np.random.normal(size=1000, scale=1.0), hist=False, label = "std = 1.0")

plt.subplot(2, 2, 3)
sns.distplot(np.random.normal(size=1000, scale=2.0), hist=False, label = "std = 2.0")

plt.subplot(2, 2, 4)
sns.distplot(np.random.normal(size=1000, scale=3.0), hist=False, label = "std = 3.0")

plt.show()

## Сохранение изображения <a name="save"></a>

In [ ]:
from pylab import savefig

In [ ]:
plt.subplot(2, 1, 1)
sns.distplot(np.random.normal(size=1000, scale=1.0), hist=False, label = "std = 1.0")

plt.subplot(2, 2, 3)
sns.distplot(np.random.normal(size=1000, scale=2.0), hist=False, label = "std = 2.0")

plt.subplot(2, 2, 4)
sns.distplot(np.random.normal(size=1000, scale=3.0), hist=False, label = "std = 3.0")

savefig('foo.pdf') # vectorized
savefig('foo.png') # rasterized
# чтобы удалить белые поля по бокам изображения, передайте аргумент bbox_inches='tight'

## Подписи осей <a name="labels"></a>

На многих графиках выше одна из осей или даже обе оставались без подписей. В таком случае сложно понять, что именно изображено на графике. Не надо так.  
Подписи можно задать с помощью функций `xlabel` и `ylabel`.

In [ ]:
sns.distplot(np.random.normal(size=1000, scale=1.0), hist=False, label = "std = 1.0")
sns.distplot(np.random.normal(size=1000, scale=2.0), hist=False, label = "std = 2.0")
sns.distplot(np.random.normal(size=1000, scale=3.0), hist=False, label = "std = 3.0")

plt.xlabel('x')
plt.ylabel('f(x)')

plt.title("N(0, std)")

plt.legend(loc='best')
plt.show()